In [1]:
from Plain import Dissertation_Plain_1D

In [4]:
# run_speeds_plain_L200N20001.py
import os, json
from pathlib import Path
import numpy as np
from joblib import Parallel, delayed

# ========= helpers for saving =========
def _ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def _atomic_write_json(path: Path, obj):
    tmp = path.with_suffix(".tmp")
    with open(tmp, "w") as f:
        json.dump(obj, f, indent=2)
    os.replace(tmp, path)

def _save_summary(run_dir: Path, meta: dict):
    with open(run_dir / "summary.json", "w") as f:
        json.dump(meta, f, indent=2)

def _save_fronts(run_dir: Path, t_fronts, x_fronts):
    # Backward-compatible: saves the default (N) fronts
    np.savez_compressed(run_dir / "fronts.npz",
                        t_fronts=np.asarray(t_fronts),
                        x_fronts=np.asarray(x_fronts))

def _save_fronts_named(run_dir: Path, name: str, t_fronts, x_fronts):
    np.savez_compressed(run_dir / f"fronts_{name}.npz",
                        t_fronts=np.asarray(t_fronts),
                        x_fronts=np.asarray(x_fronts))

def _save_snapshots_every_150(run_dir: Path, model):
    # every 150th step, plus the final step
    idx = np.unique(np.concatenate([
        np.arange(0, model.Nt, 150),
        np.array([model.Nt - 1])
    ]))
    np.savez_compressed(
        run_dir / "snapshots.npz",
        x=model.x,
        times=model.times[idx],
        N_arr=model.N_arr[idx, :],
        M_arr=model.M_arr[idx, :]
    )

# ========= single run worker (stores N and optional M speeds) =========
def run_one(lam, m0,
            base_dir="speeds_plain_L200N20001",
            model_kwargs=None,
            overwrite=False):
    if model_kwargs is None:
        model_kwargs = {}

    try:
        base = Path(base_dir)
        lam_dir = base / f"lambda_{lam}"
        run_dir = lam_dir / f"m0_{m0}"
        _ensure_dir(run_dir)

        # Skip if exists (unless overwrite)
        if not overwrite and (run_dir / "summary.json").exists():
            return ("skipped", lam, m0)

        # Avoid thread over-subscription
        os.environ.setdefault("OMP_NUM_THREADS", "1")
        os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
        os.environ.setdefault("MKL_NUM_THREADS", "1")
        os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

        # Build & solve
        model = Dissertation_Plain_1D(k=lam, m0=m0, **model_kwargs)
        model.solve()

        # ===== N-front metrics =====
        c_N, b_N, r2_N = model.estimate_wave_speed(
            threshold=0.5, band=(0.1, 0.9), spline_type='cubic',
            plot=False, target='N'
        )
        if (c_N is None) or (isinstance(c_N, float) and np.isnan(c_N)):
            raise ValueError("Wave speed (N) could not be calculated")
        model.wave_speed = c_N

        # Front tracking for N
        tN, xN = model.track_wavefront_local_interpolation(
            threshold=0.5, band=(0.1, 0.9),
            spline_type='cubic', target='N'
        )

        # ===== M-front metrics (optional) =====
        # Use m0/2 as threshold; if m0<=0, skip
        m_threshold = 0.5 * float(m0)
        c_M, r2_M, tM, xM = None, None, None, None
        if m_threshold > 0.0:
            try:
                c_M, b_M, r2_M = model.estimate_wave_speed(
                    threshold=m_threshold, band=(0.1, 0.9),
                    spline_type='cubic', plot=False, target='M'
                )
                if (c_M is not None) and not (isinstance(c_M, float) and np.isnan(c_M)):
                    tM, xM = model.track_wavefront_local_interpolation(
                        threshold=m_threshold, band=(0.1, 0.9),
                        spline_type='cubic', target='M'
                    )
            except Exception:
                pass

        # Save artifacts
        summary = dict(
            lambda_val=lam, m0=m0,
            # N-front
            wave_speed=float(c_N),
            r2=(float(r2_N) if r2_N is not None else None),
            # M-front
            m_threshold=m_threshold,
            m_wave_speed=(float(c_M) if (c_M is not None and not (isinstance(c_M, float) and np.isnan(c_M))) else None),
            m_r2=(float(r2_M) if r2_M is not None else None),
            # meta
            dt=model.dt, T=model.T, L=model.L, N=model.N,
            init_type=model.init_type,
            t_start=model.t_start, t_end=model.t_end,
            saved_stride=150
        )
        _save_summary(run_dir, summary)

        _save_fronts(run_dir, tN, xN)
        _save_fronts_named(run_dir, "N", tN, xN)
        if (tM is not None) and (xM is not None):
            _save_fronts_named(run_dir, "M", tM, xM)

        _save_snapshots_every_150(run_dir, model)

        return ("done", lam, m0,
                float(c_N), (float(r2_N) if r2_N is not None else None),
                (float(c_M) if c_M is not None else None),
                (float(r2_M) if r2_M is not None else None))

    except Exception as e:
        return ("failed", lam, m0, str(e))

# ========= grid runner with failure lists =========
def run_grid(lambda_vals, m0_vals,
             base_dir="speeds_plain_L200N20001",
             model_kwargs=None,
             overwrite=False,
             n_jobs=-1, verbose=10):
    if model_kwargs is None:
        model_kwargs = {}

    tasks = [(lam, m0) for lam in lambda_vals for m0 in m0_vals]
    results = Parallel(n_jobs=n_jobs, verbose=verbose, backend="loky")(
        delayed(run_one)(
            lam, m0,
            base_dir=base_dir,
            model_kwargs=model_kwargs,
            overwrite=overwrite
        ) for lam, m0 in tasks
    )

    done, skipped, failed, low_r2_N, low_r2_M = [], [], [], [], []
    for r in results:
        tag = r[0]
        if tag == "done":
            _, lam, m0, cN, r2N, cM, r2M = r
            done.append({"lambda": lam, "m0": m0, "cN": cN, "r2N": r2N, "cM": cM, "r2M": r2M})
            if (r2N is None) or (isinstance(r2N, float) and (np.isnan(r2N) or r2N < .999)):
                low_r2_N.append({"lambda": lam, "m0": m0, "cN": cN, "r2N": r2N})
            if (cM is not None):
                if (r2M is None) or (isinstance(r2M, float) and (np.isnan(r2M) or r2M < .999)):
                    low_r2_M.append({"lambda": lam, "m0": m0, "cM": cM, "r2M": r2M})
        elif tag == "skipped":
            _, lam, m0 = r
            skipped.append({"lambda": lam, "m0": m0})
        elif tag == "failed":
            _, lam, m0, msg = r
            failed.append({"lambda": lam, "m0": m0, "error": msg})

    base = Path(base_dir); _ensure_dir(base)
    _atomic_write_json(base / "failed_runs.json", failed)
    _atomic_write_json(base / "low_r2_runs_N.json", low_r2_N)
    _atomic_write_json(base / "low_r2_runs_M.json", low_r2_M)

    print(f"✅ Done: {len(done)}, Skipped: {len(skipped)}, Failed: {len(failed)}, Low-R² (N): {len(low_r2_N)}, Low-R² (M): {len(low_r2_M)}")
    if failed:
        print("❌ Failed runs:")
        for item in failed[:10]:
            print(f"  λ={item['lambda']}, m0={item['m0']} | error: {item['error']}")
    if low_r2_N:
        print("⚠️  Low-R² (N) examples:")
        for item in low_r2_N[:10]:
            print(f"  λ={item['lambda']}, m0={item['m0']} | cN={item['cN']} | r2N={item['r2N']}")
    if low_r2_M:
        print("⚠️  Low-R² (M) examples:")
        for item in low_r2_M[:10]:
            print(f"  λ={item['lambda']}, m0={item['m0']} | cM={item['cM']} | r2M={item['r2M']}")

    return {"done": done, "skipped": skipped, "failed": failed,
            "low_r2_N": low_r2_N, "low_r2_M": low_r2_M}

In [5]:
# =========================
# ========== RUN ==========
# =========================

# Lambda sweep (as requested)
lambda_vals = [0.001, 0.01, 0.05, 1, 5, 10, 100, 1000, 1e5, 1e6]
# Choose m0 sweep (edit as you like; keeping 0 and 1 out)
m0_vals = [0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5,
       0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]
# Shared model args for this batch
shared_kwargs = dict(
    L=200, N=20001, T=200, dt=0.1,
    init_type="tanh", steepness=0.85, perc=0.4,
    t_start=60, t_end=180, num_points=200,
    n0=1.0, K=1.0, rho=1.0, D=1.0, Mmax=1.0
)
results = run_grid(lambda_vals, m0_vals,
                   base_dir="speeds_plain_L200N20001",
                   model_kwargs=shared_kwargs,
                   overwrite=False,
                   n_jobs=-1, verbose=10)
# Optional: print a quick flag summary from saved summaries
print("\n=== Quick check for problematic runs (from saved summaries) ===")
flagged = []
def _flag_from_summary(meta, lam, m0):
    cN = meta.get("wave_speed", None); r2N = meta.get("r2", None)
    if cN is None:
        flagged.append({"lambda": lam, "m0": m0, "reason": "NO N-SPEED"})
    elif (r2N is None) or (isinstance(r2N, float) and (np.isnan(r2N) or r2N < 0.999)):
        flagged.append({"lambda": lam, "m0": m0, "reason": f"LOW N R² ({r2N})"})
    cM = meta.get("m_wave_speed", None); r2M = meta.get("m_r2", None)
    mthr = meta.get("m_threshold", None)
    if mthr is not None and mthr > 0:
        if cM is None:
            flagged.append({"lambda": lam, "m0": m0, "reason": "NO M-SPEED"})
        elif (r2M is None) or (isinstance(r2M, float) and (np.isnan(r2M) or r2M < 0.999)):
            flagged.append({"lambda": lam, "m0": m0, "reason": f"LOW M R² ({r2M})"})
base = Path("speeds_plain_L200N20001")
for lam in lambda_vals:
    for m0 in m0_vals:
        s = base / f"lambda_{lam}" / f"m0_{m0}" / "summary.json"
        if s.exists():
            with open(s, "r") as f:
                meta = json.load(f)
            _flag_from_summary(meta, lam, m0)
if flagged:
    for item in flagged:
        print(f"λ={item['lambda']}, m0={item['m0']} -> {item['reason']}")
else:
    print("✅ No failures, all available speeds calculated with high R².")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Estimated speed = 0.9220, R² = 1.0000
Estimated speed = 1.0007, R² = 1.0000
❌ Not enough valid front points.
Estimated speed = 0.2015, R² = 0.9999
Estimated speed = 0.8078, R² = 1.0000
Estimated speed = 0.4407, R² = 0.9999
❌ Not enough valid front points.
Estimated speed = 0.9815, R² = 1.0000
❌ Not enough valid front points.
❌ Not enough valid front points.
Estimated speed = 0.6055, R² = 1.0000


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.7min


❌ Not enough valid front points.
❌ Not enough valid front points.
❌ Not enough valid front points.
❌ Not enough valid front points.
Estimated speed = 0.8078, R² = 1.0000
Estimated speed = 0.9220, R² = 1.0000
Estimated speed = 0.9815, R² = 1.0000
Estimated speed = 0.2015, R² = 0.9999
Estimated speed = 0.0162, R² = 1.0000
Estimated speed = 0.4407, R² = 0.9999
Estimated speed = 0.6055, R² = 1.0000
❌ Not enough valid front points.
❌ Not enough valid front points.
❌ Not enough valid front points.
❌ Not enough valid front points.
❌ Not enough valid front points.
❌ Not enough valid front points.
❌ Not enough valid front points.


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.3min


...

[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 23.6min


Estimated speed = 0.9978, R² = 1.0000
Estimated speed = 0.9976, R² = 1.0000
Estimated speed = 0.9797, R² = 1.0000
Estimated speed = 0.9233, R² = 1.0000
Estimated speed = 0.9790, R² = 1.0000
Estimated speed = 0.9221, R² = 1.0000
Estimated speed = 0.6757, R² = 1.0000
Estimated speed = 0.7452, R² = 1.0000
Estimated speed = 0.6755, R² = 1.0000
Estimated speed = 0.7452, R² = 1.0000
Estimated speed = 0.8392, R² = 1.0000
Estimated speed = 0.8389, R² = 1.0000
Estimated speed = 0.5473, R² = 1.0000
Estimated speed = 0.5473, R² = 1.0000
Estimated speed = 0.1640, R² = 1.0000
Estimated speed = 0.1640, R² = 1.0000
❌ Not enough valid front points.
Estimated speed = 0.1992, R² = 1.0000
❌ Not enough valid front points.
Estimated speed = 0.4360, R² = 1.0000
❌ Not enough valid front points.
Estimated speed = 0.7992, R² = 1.0000
❌ Not enough valid front points.
Estimated speed = 0.5995, R² = 1.0000
❌ Not enough valid front points.
Estimated speed = 0.9141, R² = 1.0000
Estimated speed = 0.9126, R² = 1.0000

[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 27.5min


Estimated speed = 0.9206, R² = 1.0000
Estimated speed = 0.9189, R² = 1.0000
Estimated speed = 0.8389, R² = 1.0000
Estimated speed = 0.8387, R² = 1.0000
Estimated speed = 0.6761, R² = 1.0000
Estimated speed = 0.7456, R² = 1.0000
Estimated speed = 0.7455, R² = 1.0000
Estimated speed = 0.6760, R² = 1.0000
Estimated speed = 0.5478, R² = 1.0000
Estimated speed = 0.5478, R² = 1.0000
Estimated speed = 0.1642, R² = 1.0000
Estimated speed = 0.1642, R² = 1.0000
❌ Not enough valid front points.
Estimated speed = 0.1988, R² = 1.0000
❌ Not enough valid front points.
Estimated speed = 0.4351, R² = 1.0000
❌ Not enough valid front points.
Estimated speed = 0.7976, R² = 1.0000
Estimated speed = 0.5983, R² = 1.0000
Estimated speed = 0.9124, R² = 1.0000
❌ Not enough valid front points.
❌ Not enough valid front points.
Estimated speed = 0.9110, R² = 1.0000
Estimated speed = 0.9742, R² = 1.0000
Estimated speed = 0.9734, R² = 1.0000
Estimated speed = 0.9757, R² = 1.0000
Estimated speed = 0.9749, R² = 1.0000

[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 30.1min finished
